In [1]:
import tensorflow as tf
import time
from datetime import timedelta
import math
import random
import numpy as np
import dataset

#Adding Seed so that random initialization is consistent
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)


batch_size = 32

#Prepare input data
classes = ['dogs','cats']
num_classes = len(classes)

# 20% of the data will automatically be used for validation
validation_size = 0.2
img_size = 128
num_channels = 3
train_path='training_data'

# We shall load all the training and validation images and labels into memory using openCV and use that during training
data = dataset.read_train_sets(train_path, img_size, classes, validation_size=validation_size)


print("Complete reading input data. Will Now print a snippet of it")
print("Number of files in Training-set:\t\t{}".format(len(data.train.labels)))
print("Number of files in Validation-set:\t{}".format(len(data.valid.labels)))


session = tf.Session()
X = tf.placeholder(tf.float32, shape=[None, img_size,img_size,num_channels], name='x')

## labels
Y_true = tf.placeholder(tf.float32, shape=[None, 2], name='y_true')
Y_true_cls = tf.argmax(Y_true, dimension=1)

global_step =10

def create_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def create_biases(shape):
    return tf.Variable(tf.constant(0.05, shape=shape))

def conv2d(X,W):
    return tf.mm.conv2d(X,W,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(X):
    return tf.nn.max_pool(X,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')


Going to read training images
Now going to read dogs files (Index: 0)
Now going to read cats files (Index: 1)
Complete reading input data. Will Now print a snippet of it
Number of files in Training-set:		800
Number of files in Validation-set:	200
Instructions for updating:
Use the `axis` argument instead


In [2]:
W_conv1=create_weights([3,3,3,32])
b_conv1=create_biases([32])

In [3]:
layer1=tf.nn.conv2d(X,W_conv1,strides=[1,1,1,1],padding='SAME')
layer1= layer1 + b_conv1

In [4]:
layer_max_pool1=tf.nn.max_pool(layer1,ksize=[1,2,2,1],strides =[1,2,2,1],padding='SAME')

In [5]:
layer_relu1=tf.nn.relu(layer_max_pool1)

In [6]:
W_conv2=create_weights([3,3,32,32])
b_conv2=create_biases([32])

In [7]:
layer2=tf.nn.conv2d(layer_relu1,W_conv2,strides=[1,1,1,1],padding='SAME')
layer2= layer2+b_conv2

In [8]:
layer_max_pool2=tf.nn.max_pool(layer2,ksize=[1,2,2,1],strides =[1,2,2,1],padding='SAME')

In [9]:
layer_relu2=tf.nn.relu(layer_max_pool2)

In [10]:
W_conv3=create_weights([3,3,32,64])
b_conv3=create_biases([64])
layer3=tf.nn.conv2d(layer_relu2,W_conv3,strides=[1,1,1,1],padding='SAME')
layer3= layer3+b_conv3
layer_max_pool3=tf.nn.max_pool(layer3,ksize=[1,2,2,1],strides =[1,2,2,1],padding='SAME')
layer_relu3=tf.nn.relu(layer_max_pool3)
print(layer_relu3.shape)

(?, 16, 16, 64)


In [11]:
fc1_flat = tf.reshape (layer_relu3,[-1,16*16*64])
print(fc1_flat.shape)

(?, 16384)


In [12]:
weights_fc1 = create_weights([fc1_flat.get_shape()[1:4].num_elements(),128])
print(weights_fc1.shape)
biases_fc1= create_weights([128])

(16384, 128)


In [13]:
fc1 = tf.matmul(fc1_flat, weights_fc1) + biases_fc1
print(fc1.shape)

(?, 128)


In [14]:
weights_fc2 = create_weights([fc1.get_shape()[1:4].num_elements(),2])
biases_fc2= create_weights([2])

In [15]:
fc2 = tf.matmul(fc1, weights_fc2) + biases_fc2

In [16]:
y_pred =tf.nn.softmax(fc2, name='y_pred')
y_pred_cls=tf.argmax(y_pred, dimension=1)

In [17]:
session.run(tf.global_variables_initializer())

In [18]:
cross_entropy= tf.nn.softmax_cross_entropy_with_logits(logits=fc2, labels=Y_true)

In [19]:
cost=tf.reduce_mean(cross_entropy)
optimizer=tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

In [20]:
correct_prediction=tf.equal(y_pred_cls,Y_true_cls)
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [ ]:
train_path='training_data'


# Merge all summaries
#merged_summary_op  = tf.summary.merge_all()

session.run(tf.global_variables_initializer()) 

batch_size = 32
total_iterations = 0
train_batch_size = 32

x_batch, y_true_batch, _, cls_batch = data.train.next_batch(batch_size)

feed_dict_train = {X: x_batch, Y_true: y_true_batch}

session.run(optimizer, feed_dict=feed_dict_train)


x_valid_batch, y_valid_batch, _, valid_cls_batch = data.valid.next_batch(train_batch_size)

feed_dict_val = {X: x_valid_batch, Y_true: y_valid_batch}

val_loss = session.run(cost, feed_dict=feed_dict_val)

def show_progress(epoch, feed_dict_train, feed_dict_validate, val_loss):
    acc = session.run(accuracy, feed_dict=feed_dict_train)
    val_acc = session.run(accuracy, feed_dict=feed_dict_validate)
    msg = "Training Epoch {0} --- Training Accuracy: {1:>6.1%}, Validation Accuracy: {2:>6.1%},  Validation Loss: {3:.3f}"
    print(msg.format(epoch + 1, acc, val_acc, val_loss))


def train(num_iteration):
    global total_iterations

    for i in range(total_iterations, total_iterations + num_iteration):

        x_batch, y_true_batch, _, cls_batch = data.train.next_batch(batch_size)
        x_valid_batch, y_valid_batch, _, valid_cls_batch = data.valid.next_batch(batch_size)

        feed_dict_tr = {X: x_batch, Y_true: y_true_batch}
        feed_dict_val = {X: x_valid_batch, Y_true: y_valid_batch}

        session.run(optimizer, feed_dict=feed_dict_tr)

        if i % int(data.train.num_examples / batch_size) == 0:
            val_loss = session.run(cost, feed_dict=feed_dict_val)
            epoch = int(i / int(data.train.num_examples / batch_size))

            show_progress(epoch, feed_dict_tr, feed_dict_val, val_loss)
            #saver.save(session, 'dogs-cats-model')

    total_iterations += num_iteration

 
train(num_iteration=3000)


Training Epoch 1 --- Training Accuracy:  53.1%, Validation Accuracy:  46.9%,  Validation Loss: 0.739
Training Epoch 2 --- Training Accuracy:  65.6%, Validation Accuracy:  56.2%,  Validation Loss: 0.689
Training Epoch 3 --- Training Accuracy:  56.2%, Validation Accuracy:  53.1%,  Validation Loss: 0.683
Training Epoch 4 --- Training Accuracy:  65.6%, Validation Accuracy:  59.4%,  Validation Loss: 0.687
Training Epoch 5 --- Training Accuracy:  71.9%, Validation Accuracy:  43.8%,  Validation Loss: 0.746
Training Epoch 6 --- Training Accuracy:  62.5%, Validation Accuracy:  43.8%,  Validation Loss: 0.736
Training Epoch 7 --- Training Accuracy:  62.5%, Validation Accuracy:  56.2%,  Validation Loss: 0.672
Training Epoch 8 --- Training Accuracy:  62.5%, Validation Accuracy:  68.8%,  Validation Loss: 0.625
Training Epoch 9 --- Training Accuracy:  81.2%, Validation Accuracy:  62.5%,  Validation Loss: 0.655
Training Epoch 10 --- Training Accuracy:  81.2%, Validation Accuracy:  65.6%,  Validation L